# import

In [23]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
import os, random
import csv
from tqdm import tqdm

SAVE_DIR = "./data/"
save_file_name = "conversation_raw.csv"
file_path = os.path.join(SAVE_DIR, save_file_name)

# 環境変数の設定
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY_IKEG")

# perosonality_prompt

In [33]:
n_agents = 7

personality_prompt = f"""
Task:
Your task is to determine the personality of {n_agents} agent. This personalities influences the values of the actions taken by the agent.

Guidelines:
1. output should be not too long, about 10 words.
2. Do NOT write "high vengefulness and low boldness" expressly in outline.
3. #Agent1, #Agent2... and so on, separated by agent.
4. The behavior patterns should be as diverse as possible for each agent in terms of possible strategies.
5. Do not write unnecessary explanations.
6. The each outputs includes short description of "Personality Traits:", but this doens't count in the 10 words limit.
"""

In [52]:
gpt4 = ChatOpenAI(model_name = "gpt-4-0125-preview", temperature=0.7, model_kwargs={"top_p":0.9})

output = gpt4.predict(personality_prompt)

print(output)

#Agent1
Personality Traits: Cautious, strategic, values alliances, avoids unnecessary risks.

#Agent2
Personality Traits: Aggressive, highly competitive, seeks dominance, disregards alliances.

#Agent3
Personality Traits: Charismatic, manipulative, forms alliances, betrays for gain.

#Agent4
Personality Traits: Analytical, patient, waits for opportune moments, risk-averse.

#Agent5
Personality Traits: Impulsive, unpredictable, acts on whims, ignores strategy.

#Agent6
Personality Traits: Altruistic, cooperative, prioritizes group success, avoids conflict.

#Agent7
Personality Traits: Inventive, experimental, tries unconventional strategies, learns from failures.


In [53]:
import pandas as pd

def extract_info_and_save_to_csv(text, csv_path):
    sections = ["Personality Traits"]
    agents = text.split("#")[1:]  # 最初の要素は空なのでスキップ

    data = []
    for agent in agents:
        lines = agent.split("\n")
        agent_info = {}

        for section in sections:
            for line in lines:
                if section in line:
                    # セクション名と内容を分離し、辞書に追加
                    content = line.split(":")[1].strip()
                    agent_info[section] = content
                    break  # 同じセクションに再び遭遇しないようにループを抜ける

        data.append(agent_info)

    # CSVファイルの追加または作成部
    if os.path.exists(csv_path):
        df_existing = pd.read_csv(csv_path)
        df_new = pd.DataFrame(data)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    else:
        df_combined = pd.DataFrame(data)

    df_combined.columns = sections
    df_combined.to_csv(csv_path, index=False)
    return csv_path

In [54]:
csv_path = "./data/agents_character_v2.csv"
append_or_create_path = extract_info_and_save_to_csv(output, csv_path)
append_or_create_path

'./data/agents_character_v2.csv'